In [2]:
import pandas as pd 
import numpy as np
import pymongo
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

In [3]:
#import data from mongodb
client = pymongo.MongoClient()
db = client.covid19
collection = db.data

In [4]:
#creates dataframe withouth the mongodb _id column
df = pd.DataFrame(list(collection.find())).iloc[:,1:]

In [5]:
#updated datatype to datetime
df[['Last Update', 'Last_Update']] = df[['Last Update', 'Last_Update']].astype('datetime64[ns]')
df[['Confirmed','Deaths','Recovered']].fillna(0, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 17 columns):
Province/State    4358 non-null object
Country/Region    7617 non-null object
Last Update       7617 non-null datetime64[ns]
Confirmed         34981 non-null float64
Deaths            34559 non-null float64
Recovered         34612 non-null float64
FIPS              25196 non-null float64
Admin2            25365 non-null object
Province_State    26036 non-null object
Country_Region    27383 non-null object
Last_Update       27383 non-null datetime64[ns]
Lat               27370 non-null float64
Long_             27370 non-null float64
Active            27383 non-null float64
Combined_Key      27383 non-null object
Latitude          4799 non-null float64
Longitude         4799 non-null float64
dtypes: datetime64[ns](2), float64(9), object(6)
memory usage: 4.5+ MB


In [7]:
df.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Active,Combined_Key,Latitude,Longitude
0,Hubei,Mainland China,2020-02-26 14:13:10,65187.0,2615.0,20969.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,Guangdong,Mainland China,2020-02-26 10:33:02,1347.0,7.0,851.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,Henan,Mainland China,2020-02-26 10:33:02,1271.0,19.0,1033.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,South Korea,2020-02-26 11:03:11,1261.0,12.0,22.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,Zhejiang,Mainland China,2020-02-26 10:33:02,1205.0,1.0,867.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


# Relabeled Columns

The early csv files were named differently and had fewer columns than the current way John Hopkins is releasing its daily COVID-19 updates.  Therefore, I have made a uniform file.  Below, you can see the code and how it's consolidating the data.

In [8]:
def find_replace_na(data, main_val, replace_val, multiple=False):
    '''
    Finds and replaces null values with non-null values from other column(s) assigned to replace_val parameter(s).
    
    Parameters:
    -----------
    data : dataframe
        a pandas dataframe to be used
        
    main_val : single or list-like
        can be int, str, bool datatype, will be used as primary column(s)
        
    replace_val : single or list-like
        can be int, str, bool datatype, will be used to replace any missing value of primary column(s)
        
    multiple : bool (optional)
        defualt to False
        if True: length of main_val and replace_val must be equal
        
    Returns:
    --------
    Dataframe with filled null values on main_val parameter, with data from replace_val parameter.
    '''
    
    if multiple == False:
        data[main_val] = data[main_val].where(~pd.isnull(data[main_val]), data[replace_val])
        
    else:
        if len(main_val) == len(replace_val):
            for val in zip(main_val, replace_val):
                find_replace_na(data, val[0], val[1])
        else:
            raise BaseException('Lists provided must be same length')

    return data

In [9]:
def fillna_combine_key_values(data):
    '''
    Fills missing values from the `Combined_Key` column with non-missing values from: 
    `Admin2`, `Province_State`, and `Country_Region`
    
    Parameters:
    -----------
    data : dataframe
        a pandas dataframe to be used
        
    Returns:
    --------
    Dataframe with filled null values on `Combined_Key` column. 
    '''
    #assings columns to be used
    cols = ['Admin2', 'Province_State', 'Country_Region', 'Combined_Key']
    #converts dataframe to a list
    data_list = data[cols].values.tolist()
    
    for k, val in enumerate(data_list):
        if str(val[3]) == 'nan' and str(val[0]) !='nan' and str(val[1])!='nan':
            val[3] = str(val[0])+', '+str(val[1])+', '+str(val[2])
        elif str(val[3]) == 'nan' and str(val[1]) !='nan':
            val[3] = str(val[1])+', '+str(val[2])
        elif str(val[3]) == 'nan' and str(val[1]) =='nan':
            val[3] = val[2]
    
    data = data.fillna(pd.DataFrame(data_list, columns=cols))
    
    return data

In [10]:
def fillna_lat_lon(data):
    '''
    Fills missing values from `Latitude` and `Longitude` columns with non-missing values from same columns: 
    
    Parameters:
    -----------
    data : dataframe
        a pandas dataframe to be used
        
    Returns:
    --------
    Dataframe with filled null values on `Latitude` and `Longitude` columns. 
    '''
    lat_lon = ['Combined_Key', 'Latitude', 'Longitude']
    val_list = data[lat_lon].values.tolist()

    vals = data[lat_lon].dropna(axis=0, subset=['Latitude', 'Longitude']).reset_index(drop=True)

    for k, val in enumerate(val_list):
        if vals['Combined_Key'].any() == str(val[0]) and str(val[1])=='nan' or str(val[2])=='nan':
            val[1] = vals['Latitude'].values[0]
            val[2] = vals['Longitude'].values[0]

    data = data.fillna(pd.DataFrame(val_list, columns=lat_lon))
    
    return data

In [11]:
#these are the columns that changed over time
main_values = ['Province_State','Country_Region','Last_Update', 'Latitude', 'Longitude']
replace_values = ['Province/State','Country/Region','Last Update', 'Lat', 'Long_']

In [12]:
#consolidated values from the early reported files to the current format columns and drop old columns
updated_df = find_replace_na(df, main_values, replace_values, multiple=True)
updated_df.drop(columns=replace_values, inplace=True)

In [13]:
#updates the `Combined_Key` column to current reporting methods
updated_df = fillna_combine_key_values(updated_df)

In [14]:
#updates `Latitude`, `Longitude` values where missing, in early reporting files
updated_df = fillna_lat_lon(updated_df)

In [15]:
updated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 12 columns):
Confirmed         34981 non-null float64
Deaths            34559 non-null float64
Recovered         34612 non-null float64
FIPS              25196 non-null float64
Admin2            25365 non-null object
Province_State    30394 non-null object
Country_Region    35000 non-null object
Last_Update       35000 non-null datetime64[ns]
Active            27383 non-null float64
Combined_Key      35000 non-null object
Latitude          35000 non-null float64
Longitude         35000 non-null float64
dtypes: datetime64[ns](1), float64(7), object(4)
memory usage: 3.2+ MB


In [16]:
updated_df.head()

,Confirmed,Deaths,Recovered,FIPS,Admin2,Province_State,Country_Region,Last_Update,Active,Combined_Key,Latitude,Longitude
0,65187.0,2615.0,20969.0,NaN,NaN,Hubei,Mainland China,2020-02-26 14:13:10,NaN,"Hubei, Mainland China",34.223334,-82.461707
1,1347.0,7.0,851.0,NaN,NaN,Guangdong,Mainland China,2020-02-26 10:33:02,NaN,"Guangdong, Mainland China",34.223334,-82.461707
2,1271.0,19.0,1033.0,NaN,NaN,Henan,Mainland China,2020-02-26 10:33:02,NaN,"Henan, Mainland China",34.223334,-82.461707
3,1261.0,12.0,22.0,NaN,NaN,NaN,South Korea,2020-02-26 11:03:11,NaN,South Korea,34.223334,-82.461707
4,1205.0,1.0,867.0,NaN,NaN,Zhejiang,Mainland China,2020-02-26 10:33:02,NaN,"Zhejiang, Mainland China",34.223334,-82.461707


In [17]:
#creates date columns
updated_df['Month'] = updated_df['Last_Update'].dt.month
updated_df['Day'] = updated_df['Last_Update'].dt.day
updated_df['Year'] = updated_df['Last_Update'].dt.year
updated_df['Dayofweek'] = updated_df['Last_Update'].dt.weekday_name

In [18]:
updated_df.head()

,Confirmed,Deaths,Recovered,FIPS,Admin2,Province_State,Country_Region,Last_Update,Active,Combined_Key,Latitude,Longitude,Month,Day,Year,Dayofweek
0,65187.0,2615.0,20969.0,NaN,NaN,Hubei,Mainland China,2020-02-26 14:13:10,NaN,"Hubei, Mainland China",34.223334,-82.461707,2,26,2020,Wednesday
1,1347.0,7.0,851.0,NaN,NaN,Guangdong,Mainland China,2020-02-26 10:33:02,NaN,"Guangdong, Mainland China",34.223334,-82.461707,2,26,2020,Wednesday
2,1271.0,19.0,1033.0,NaN,NaN,Henan,Mainland China,2020-02-26 10:33:02,NaN,"Henan, Mainland China",34.223334,-82.461707,2,26,2020,Wednesday
3,1261.0,12.0,22.0,NaN,NaN,NaN,South Korea,2020-02-26 11:03:11,NaN,South Korea,34.223334,-82.461707,2,26,2020,Wednesday
4,1205.0,1.0,867.0,NaN,NaN,Zhejiang,Mainland China,2020-02-26 10:33:02,NaN,"Zhejiang, Mainland China",34.223334,-82.461707,2,26,2020,Wednesday


In [19]:
updated_df.to_csv('../data/cleaned_csse_covid_19_dataset.csv', index=False)